In [1]:
#!pip install --upgrade google-cloud-bigquery[pandas]

In [2]:
from google.cloud import bigquery

In [3]:
client=bigquery.Client()

C:\ProgramData\Anaconda3\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
%load_ext google.cloud.bigquery

In [5]:
%reload_ext google.cloud.bigquery

In [6]:
#######################################
print('Setting everything up...')
#######################################

import warnings
warnings.filterwarnings('ignore')
import pandas_gbq 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D

import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

import matplotlib.pyplot as plt




import os
import sys
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
import time

DATASET = 'aou-res-curation-prod.prod_drc_dataset'

plt.style.use('ggplot')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)


print('done.')


Setting everything up...
done.


In [7]:
dic={'src_hpo_id':["seec_miami","seec_morehouse","seec_emory","uamc_banner","pitt","nyc_cu","ipmc_uic","trans_am_spectrum","tach_hfhs","nec_bmc","cpmc_uci","nec_phs","nyc_cornell","ipmc_nu","nyc_hh","ipmc_uchicago","aouw_mcri","syhc","cpmc_ceders","seec_ufl","saou_uab","trans_am_baylor","cpmc_ucsd","ecchc","chci","aouw_uwh","cpmc_usc","hrhc","ipmc_northshore","chs","cpmc_ucsf","jhchc","aouw_mcw","cpmc_ucd","ipmc_rush"],
    'HPO':["SouthEast Enrollment Center Miami","SouthEast Enrollment Center Morehouse","SouthEast Enrollment Center Emory","Banner Health","University of Pittsburgh","Columbia University Medical Center","University of Illinois Chicago","Spectrum Health","Henry Ford Health System","Boston Medical Center","UC Irvine","Partners HealthCare","Weill Cornell Medical Center","Northwestern Memorial Hospital","Harlem Hospital","University of Chicago","Marshfield Clinic","San Ysidro Health Center","Cedars-Sinai","University of Florida","University of Alabama at Birmingham","Baylor","UC San Diego","Eau Claire Cooperative Health Center","Community Health Center, Inc.","UW Health (University of Wisconsin Madison)","University of Southern California","HRHCare","NorthShore University Health System","Cherokee Health Systems","UC San Francisco","Jackson-Hinds CHC","Medical College of Wisconsin","UC Davis","Rush University"]}


site_df=pd.DataFrame(data=dic)
site_df

,src_hpo_id,HPO
0,seec_miami,SouthEast Enrollment Center Miami
1,seec_morehouse,SouthEast Enrollment Center Morehouse
2,seec_emory,SouthEast Enrollment Center Emory
3,uamc_banner,Banner Health
4,pitt,University of Pittsburgh
5,nyc_cu,Columbia University Medical Center
6,ipmc_uic,University of Illinois Chicago
7,trans_am_spectrum,Spectrum Health
8,tach_hfhs,Henry Ford Health System
9,nec_bmc,Boston Medical Center


# There should not be duplicate rows. 

## visit_occurrence table

In [8]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
       src_hpo_id,
       person_id, visit_concept_id, visit_start_date, visit_start_datetime, visit_end_date, visit_end_datetime, visit_type_concept_id, provider_id, care_site_id, visit_source_value, visit_source_concept_id, admitting_source_concept_id, admitting_source_value, discharge_to_concept_id,
       discharge_to_source_value, preceding_visit_occurrence_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_visit_occurrence` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_visit_occurrence`) AS t2
    ON
        t1.visit_occurrence_id=t2.visit_occurrence_id
    WHERE
        t1.visit_concept_id!=0 AND t1.visit_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
707183 records received.


In [9]:
foreign_key_df.head()

,src_hpo_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id,cnt
0,aouw_uwh,100239454,9202,2014-05-06,2014-05-06 18:00:00+00:00,2014-05-06,2014-05-06 18:00:00+00:00,44818518,69,3.400000e+16,OP:Outpatient Visit,None,0,*Not Applicable:*Not Applicable,0,*Not Applicable:*Not Applicable,None,3
1,aouw_uwh,100239454,9202,2014-05-19,2014-05-19 18:00:00+00:00,2014-05-19,2014-05-19 18:00:00+00:00,44818518,1390,3.400000e+16,OP:Outpatient Visit,None,0,*Not Applicable:*Not Applicable,0,*Not Applicable:*Not Applicable,None,2
2,aouw_uwh,101000578,9202,2000-07-17,2000-07-17 18:00:00+00:00,2000-07-17,2000-07-17 18:00:00+00:00,44818518,NaN,3.400000e+16,OP:Outpatient Visit,None,0,*Not Applicable:*Not Applicable,0,*Not Applicable:*Not Applicable,None,2
3,aouw_uwh,101000578,9202,2014-10-30,2014-10-30 18:00:00+00:00,2014-10-30,2014-10-30 18:00:00+00:00,44818518,NaN,3.400000e+16,OP:Outpatient Visit,None,0,*Not Applicable:*Not Applicable,0,*Not Applicable:*Not Applicable,None,2
4,aouw_uwh,101000578,9202,2017-11-17,2017-11-17 18:00:00+00:00,2017-11-17,2017-11-17 18:00:00+00:00,44818518,3221,3.400000e+16,OP:Outpatient Visit,None,0,*Not Applicable:*Not Applicable,0,*Not Applicable:*Not Applicable,None,2


In [10]:
visit_occurrence=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'visit_occurrence'}).sort_values(["visit_occurrence"]).set_index("src_hpo_id")
visit_occurrence=visit_occurrence.reset_index()
visit_occurrence

,src_hpo_id,visit_occurrence
0,ecchc,5
1,chci,59
2,tach_hfhs,270
3,nyc_hh,396
4,pitt,786
5,ipmc_northshore,814
6,syhc,1113
7,cpmc_usc,1401
8,uamc_banner,2099
9,saou_uab,2465


## condition_occurrence table

In [11]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
     src_hpo_id,
person_id, condition_concept_id, condition_start_date, condition_start_datetime, condition_end_date,
condition_end_datetime, condition_type_concept_id, stop_reason, provider_id, visit_occurrence_id,
condition_source_value, condition_source_concept_id, condition_status_source_value, condition_status_concept_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_condition_occurrence` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_condition_occurrence`) AS t2
    ON
        t1.condition_occurrence_id=t2.condition_occurrence_id
    WHERE
        t1.condition_concept_id!=0 AND t1.condition_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
1000331 records received.


In [12]:
foreign_key_df.head()

,src_hpo_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id,cnt
0,aouw_uwh,100239454,200219,2014-05-05,2014-05-05 18:00:00+00:00,2014-05-05,2014-05-05 18:00:00+00:00,32019,None,69.0,3.400000e+16,21522001:Ureteral colic,200219,Active:Active,0.0,2
1,aouw_uwh,100239454,320136,2013-01-09,2013-01-09 18:00:00+00:00,2013-01-09,2013-01-09 18:00:00+00:00,32019,None,278.0,3.400000e+16,50043002:Zenker diverticulum,320136,Active:Active,0.0,2
2,aouw_uwh,100239454,320136,2013-01-09,2013-01-09 18:00:00+00:00,2013-01-09,2013-01-09 18:00:00+00:00,32020,None,278.0,3.400000e+16,50043002:Zenker diverticulum,320136,Active:Active,0.0,2
3,aouw_uwh,100239454,432250,2014-05-06,2014-05-06 18:00:00+00:00,2014-05-17,2014-05-17 18:00:00+00:00,32019,None,17106.0,3.400000e+16,40733004:Zoster without complications,432250,Active:Active,0.0,3
4,aouw_uwh,100239454,432545,2014-05-06,2014-05-06 18:00:00+00:00,2014-05-17,2014-05-17 18:00:00+00:00,32019,None,17106.0,3.400000e+16,87628006:Walking pneumonia,432545,Active:Active,0.0,3


In [13]:
condition_occurrence=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'condition_occurrence'}).sort_values(["condition_occurrence"]).set_index("src_hpo_id")
condition_occurrence=condition_occurrence.reset_index()
condition_occurrence

,src_hpo_id,condition_occurrence
0,nec_phs,4
1,saou_uab,4
2,uamc_banner,22
3,nyc_hh,45
4,ipmc_rush,454
5,cpmc_uci,1132
6,cpmc_ceders,1823
7,jhchc,6321
8,cpmc_ucsd,33477
9,aouw_uwh,957049


## drug_exposure table

In [14]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        src_hpo_id,
person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_end_date, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, 
days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, drug_source_value,
drug_source_concept_id, route_source_value, dose_unit_source_value,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_drug_exposure` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_drug_exposure`) AS t2
    ON
        t1.drug_exposure_id=t2.drug_exposure_id
    WHERE
        t1.drug_concept_id!=0 AND t1.drug_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
460970 records received.


In [15]:
foreign_key_df.head()

,src_hpo_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_end_date,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,cnt
0,aouw_mcri,127101023,939510,2018-10-15,2018-10-16,NaT,581373,,0,1.0,3,,NaN,,5023728.0,3.300000e+16,SODIUM BICARBONATE (ORAL PILL),939510.0,ORAL PILL,,2
1,aouw_mcri,127101023,939510,2018-10-15,2018-10-18,NaT,38000175,,0,1.0,3,,NaN,,23728.0,3.300000e+16,SODIUM BICARBONATE (ORAL PILL),939510.0,ORAL PILL,,2
2,aouw_mcri,127101023,19134047,2018-10-12,2018-10-15,NaT,38000175,,0,1.0,3,,NaN,,23728.0,3.300000e+16,TRAMADOL (ORAL PILL),19134047.0,ORAL PILL,,3
3,aouw_mcri,127101023,19134047,2018-10-12,2018-10-13,NaT,581373,,0,1.0,3,,NaN,,5023728.0,3.300000e+16,TRAMADOL (ORAL PILL),19134047.0,ORAL PILL,,3
4,aouw_mcri,127101023,40174811,2018-10-15,2018-10-16,NaT,581373,,0,1.0,3,,NaN,,5023728.0,3.300000e+16,HYDRALAZINE (ORAL PILL),40174811.0,ORAL PILL,,4


In [16]:
drug_exposure=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'drug_exposure'}).sort_values(["drug_exposure"]).set_index("src_hpo_id")
drug_exposure=drug_exposure.reset_index()
drug_exposure

,src_hpo_id,drug_exposure
0,ecchc,1
1,chs,3
2,ipmc_northshore,8
3,nyc_hh,35
4,aouw_mcri,41
5,jhchc,79
6,cpmc_ucd,121
7,ipmc_rush,178
8,seec_miami,228
9,trans_am_spectrum,485


## measurement table

In [17]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        src_hpo_id,
person_id, measurement_concept_id, measurement_date, measurement_datetime, measurement_type_concept_id, operator_concept_id, value_as_number, value_as_concept_id, unit_concept_id, range_low, range_high, provider_id, visit_occurrence_id,
measurement_source_value, measurement_source_concept_id, unit_source_value, value_source_value,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_measurement` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_measurement`) AS t2
    ON
        t1.measurement_id=t2.measurement_id
    WHERE
        t1.measurement_concept_id!=0 AND t1.measurement_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
35464 records received.


In [18]:
foreign_key_df.head()

,src_hpo_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value,cnt
0,aouw_uwh,100239454,2212980,2014-05-13,2014-05-13 13:15:00+00:00,44818702,NaN,NaN,NaN,NaN,NaN,NaN,560.0,3.400000e+16,"GM4045:CULTURE, BLOOD, BACTERIA AND YEAST",NaN,None,None,4
1,aouw_uwh,100239454,2212980,2014-05-07,2014-05-07 18:10:00+00:00,5001,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3.400000e+16,"GM4045:CULTURE, BLOOD, BACTERIA AND YEAST",NaN,None,None,2
2,aouw_uwh,100239454,2212980,2014-05-06,2014-05-06 15:41:00+00:00,5001,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3.400000e+16,"GM4045:CULTURE, BLOOD, BACTERIA AND YEAST",NaN,None,None,2
3,aouw_uwh,100239454,2212980,2014-05-06,2014-05-06 17:32:00+00:00,5001,NaN,NaN,0.0,NaN,NaN,NaN,560.0,3.400000e+16,"GM4045:CULTURE, BLOOD, BACTERIA AND YEAST",NaN,None,None,2
4,aouw_uwh,100239454,4024510,2014-05-12,2014-05-12 09:26:00+00:00,44818702,NaN,NaN,NaN,NaN,NaN,NaN,566.0,3.400000e+16,"HCWABF:CULTURE, BODY FLUID, AER/ANA WITH GRAM STAI",NaN,None,None,32


In [19]:
measurement=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'measurement'}).sort_values(["measurement"]).set_index("src_hpo_id")
measurement=measurement.reset_index()
measurement

,src_hpo_id,measurement
0,cpmc_usc,6
1,nyc_cu,11
2,ecchc,15
3,ipmc_rush,67
4,cpmc_uci,102
5,pitt,172
6,uamc_banner,185
7,hrhc,843
8,chci,923
9,cpmc_ceders,1306


## procedure_occurrence

In [20]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        src_hpo_id,
        person_id, procedure_concept_id, procedure_date, procedure_datetime, procedure_type_concept_id, modifier_concept_id,
        quantity, provider_id, visit_occurrence_id, procedure_source_value, procedure_source_concept_id, qualifier_source_value,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_procedure_occurrence` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_procedure_occurrence`) AS t2
    ON
        t1.procedure_occurrence_id=t2.procedure_occurrence_id
    WHERE
        t1.procedure_concept_id!=0 AND t1.procedure_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
201695 records received.


In [21]:
foreign_key_df.head()

,src_hpo_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,procedure_source_value,procedure_source_concept_id,qualifier_source_value,cnt
0,aouw_mcw,100138222,2414395,2017-10-20,2017-10-20 00:00:00+00:00,0,0.0,0.0,73901.0,3.500000e+16,99211,2414395.0,None,2
1,aouw_mcw,100138222,2414395,2018-05-09,2018-05-09 00:00:00+00:00,0,0.0,0.0,85817.0,3.500000e+16,99211,2414395.0,None,2
2,aouw_mcw,101293278,2110145,2013-02-12,2013-02-12 00:00:00+00:00,0,0.0,0.0,80036.0,3.500000e+16,57455,2110145.0,None,2
3,aouw_mcw,101293278,2110155,2013-04-22,2013-04-22 00:00:00+00:00,0,0.0,0.0,11870.0,3.500000e+16,57522,2110155.0,None,3
4,aouw_mcw,101293278,2110169,2015-06-11,2015-06-11 00:00:00+00:00,0,0.0,0.0,11870.0,3.500000e+16,58100,2110169.0,None,2


In [22]:
procedure_occurrence=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'procedure_occurrence'}).sort_values(["procedure_occurrence"]).set_index("src_hpo_id")
procedure_occurrence=procedure_occurrence.reset_index()
procedure_occurrence

,src_hpo_id,procedure_occurrence
0,saou_uab,1
1,jhchc,2
2,ipmc_rush,12
3,seec_ufl,16
4,seec_emory,19
5,nyc_hh,129
6,uamc_banner,220
7,cpmc_uci,484
8,cpmc_ceders,774
9,aouw_uwh,3494


## observation table

In [23]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        src_hpo_id,
        person_id, observation_concept_id, observation_date, observation_datetime, observation_type_concept_id, value_as_number, 
        value_as_string, value_as_concept_id, qualifier_concept_id, unit_concept_id, provider_id, visit_occurrence_id, 
        observation_source_value, observation_source_concept_id, unit_source_value, qualifier_source_value, value_source_concept_id,
        value_source_value, questionnaire_response_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_observation` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_observation`) AS t2
    ON
        t1.observation_id=t2.observation_id
    WHERE
        t1.observation_concept_id!=0 AND t1.observation_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
94004 records received.


In [24]:
foreign_key_df.head()

,src_hpo_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id,visit_occurrence_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value,value_source_concept_id,value_source_value,questionnaire_response_id,cnt
0,aouw_uwh,100239454,4079646,2014-06-18,2014-06-18 18:00:00+00:00,38000280,NaN,None,0.0,None,None,19278.0,3.400000e+16,183644000:Visit for suture removal,4079646.0,None,None,None,None,None,2
1,aouw_uwh,100239454,4086921,2014-06-11,2014-06-11 18:00:00+00:00,38000280,NaN,None,0.0,None,None,19278.0,3.400000e+16,243796009:Wrist joint replacement by other means,4086921.0,None,None,None,None,None,2
2,aouw_uwh,100239454,4146314,2014-06-11,2014-06-11 18:00:00+00:00,38000280,NaN,None,0.0,None,None,19278.0,3.400000e+16,"307824009:Wound check, dressing change",4146314.0,None,None,None,None,None,3
3,aouw_uwh,100239454,4146314,2014-06-18,2014-06-18 18:00:00+00:00,38000280,NaN,None,0.0,None,None,19278.0,3.400000e+16,"307824009:Wound check, dressing change",4146314.0,None,None,None,None,None,2
4,aouw_uwh,100239454,4187630,2014-06-11,2014-06-11 18:00:00+00:00,38000280,NaN,None,0.0,None,None,19278.0,3.400000e+16,413350009:White coat syndrome without diagnosis of,4187630.0,None,None,None,None,None,2


In [25]:
observation=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'observation'}).sort_values(["observation"]).set_index("src_hpo_id")
observation=observation.reset_index()
observation

,src_hpo_id,observation
0,ipmc_rush,9
1,tach_hfhs,18
2,uamc_banner,69
3,cpmc_ceders,83
4,cpmc_uci,110
5,cpmc_ucsd,130
6,ipmc_northshore,132
7,seec_morehouse,447
8,nec_phs,2657
9,jhchc,5732


## provider table

In [26]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        provider_name, NPI, DEA, specialty_concept_id, care_site_id, year_of_birth, 
        gender_concept_id, provider_source_value, specialty_source_value, 
        specialty_source_concept_id, gender_source_value, gender_source_concept_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_provider` AS t1
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9,10,11,12
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
165074 records received.


In [27]:
foreign_key_df.head()

,provider_name,NPI,DEA,specialty_concept_id,care_site_id,year_of_birth,gender_concept_id,provider_source_value,specialty_source_value,specialty_source_concept_id,gender_source_value,gender_source_concept_id,cnt
0,,,,NaN,NaN,NaN,NaN,999999,,NaN,,NaN,2
1,,,,0.0,NaN,NaN,NaN,,,NaN,,NaN,3
2,,,,38004699.0,NaN,NaN,NaN,,,NaN,,NaN,2
3,,,,38004699.0,NaN,NaN,NaN,,,38004699.0,,NaN,2
4,(Test) MD,,,38004456.0,NaN,NaN,NaN,,,NaN,,NaN,2


## device_exposure table

In [28]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        src_hpo_id,
        person_id, device_concept_id, device_exposure_start_date, device_exposure_start_datetime, device_exposure_end_date, 
        device_exposure_end_datetime, device_type_concept_id, unique_device_id, quantity, provider_id, 
        visit_occurrence_id, device_source_value, device_source_concept_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_device_exposure` AS t1
    INNER JOIN
        (SELECT
            DISTINCT * 
    FROM
         `{}._mapping_device_exposure`) AS t2
    ON
        t1.device_exposure_id=t2.device_exposure_id
    WHERE
        t1.device_concept_id!=0 AND t1.device_concept_id IS NOT NULL AND
        t1.person_id!=0 and t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
11617 records received.


In [29]:
foreign_key_df.head()

,src_hpo_id,person_id,device_concept_id,device_exposure_start_date,device_exposure_start_datetime,device_exposure_end_date,device_exposure_end_datetime,device_type_concept_id,unique_device_id,quantity,provider_id,visit_occurrence_id,device_source_value,device_source_concept_id,cnt
0,aouw_uwh,104381362,4159813,2017-06-23,2017-06-23 18:00:00+00:00,2017-06-23,2017-06-23 18:00:00+00:00,44818707,None,NaN,NaN,3.400000e+16,C1769:GUIDE WIRE,2615779.0,2
1,aouw_uwh,104381362,4159813,2017-09-20,2017-09-20 18:00:00+00:00,2017-09-20,2017-09-20 18:00:00+00:00,44818707,None,NaN,NaN,3.400000e+16,C1769:GUIDE WIRE,2615779.0,3
2,aouw_uwh,118715143,4203382,2018-05-15,2018-05-15 11:48:00+00:00,2018-05-15,2018-05-15 11:48:00+00:00,44818707,None,NaN,NaN,3.400000e+16,EQP0017:IV PUMP (PROGRAMMING UNIT),NaN,2
3,aouw_uwh,118715143,40481859,2017-08-15,2017-08-15 18:30:00+00:00,2017-08-15,2017-08-15 18:30:00+00:00,44818707,None,NaN,NaN,3.400000e+16,EQP0023:SEQUENTIAL COMPRESSION DEVICE (SCD) / FOOT,NaN,2
4,aouw_uwh,118715143,40481859,2017-10-13,2017-10-13 09:32:00+00:00,2017-10-13,2017-10-13 09:32:00+00:00,44818707,None,NaN,NaN,3.400000e+16,EQP0023:SEQUENTIAL COMPRESSION DEVICE (SCD) / FOOT,NaN,2


In [30]:
device_exposure=foreign_key_df.groupby(['src_hpo_id']).size().reset_index().rename(columns={0:'device_exposure'}).sort_values(["device_exposure"]).set_index("src_hpo_id")
device_exposure=device_exposure.reset_index()
device_exposure

,src_hpo_id,device_exposure
0,syhc,1
1,jhchc,57
2,nyc_cornell,74
3,tach_hfhs,149
4,aouw_uwh,304
5,uamc_banner,11032


## death table

In [31]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        death_date, death_datetime, death_type_concept_id, cause_concept_id, cause_source_value, cause_source_concept_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_death` AS t1
    WHERE
        t1.death_date IS NOT NULL AND t1.person_id IS NOT NULL 
    GROUP BY
        1,2,3,4,5,6
    HAVING 
        COUNT(*) > 1    
    ORDER BY
        1,2,3,4,5,6
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
3 records received.


In [32]:
foreign_key_df.head()

,death_date,death_datetime,death_type_concept_id,cause_concept_id,cause_source_value,cause_source_concept_id,cnt
0,2018-05-20,2018-05-20 00:00:00+00:00,261,NaN,,NaN,2
1,2019-06-03,2019-06-03 00:00:00+00:00,0,0.0,None,0.0,2
2,2100-01-01,2100-01-01 07:00:00+00:00,0,0.0,None,0.0,3


## care_site table

In [33]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        place_of_service_concept_id, location_id, place_of_service_source_value,
        care_site_name, care_site_source_value,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_care_site` AS t1
    GROUP BY
        1,2,3,4,5
    HAVING 
        COUNT(*) > 1
    ORDER BY
        1,2,3,4,5
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
29906 records received.


In [34]:
foreign_key_df.head()

,place_of_service_concept_id,location_id,place_of_service_source_value,care_site_name,care_site_source_value,cnt
0,NaN,NaN,None,Peekskill Optometry,None,2
1,NaN,NaN,None,Peekskill Pediatrics,None,2
2,NaN,NaN,None,Peekskill Podiatry,None,2
3,NaN,NaN,None,Peekskill Women's Health,None,3
4,NaN,NaN,,,,6


## Sites combined

In [35]:
sites_success=pd.merge(visit_occurrence,condition_occurrence,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,drug_exposure,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,measurement,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,procedure_occurrence,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,device_exposure,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,observation,how='outer',on='src_hpo_id')
sites_success=pd.merge(sites_success,site_df,how='outer',on='src_hpo_id')

In [36]:
sites_success=sites_success.fillna(0)
sites_success[["visit_occurrence","condition_occurrence","drug_exposure","measurement","procedure_occurrence","device_exposure","observation"]]\
    =sites_success[["visit_occurrence","condition_occurrence","drug_exposure","measurement","procedure_occurrence","device_exposure","observation"]].astype(int)
sites_success

,src_hpo_id,visit_occurrence,condition_occurrence,drug_exposure,measurement,procedure_occurrence,device_exposure,observation,HPO
0,ecchc,5,0,1,15,0,0,0,Eau Claire Cooperative Health Center
1,chci,59,0,0,923,0,0,0,"Community Health Center, Inc."
2,tach_hfhs,270,0,7251,0,0,149,18,Henry Ford Health System
3,nyc_hh,396,45,35,0,129,0,0,Harlem Hospital
4,pitt,786,0,874,172,0,0,0,University of Pittsburgh
5,ipmc_northshore,814,0,8,0,0,0,132,NorthShore University Health System
6,syhc,1113,0,0,0,0,1,0,San Ysidro Health Center
7,cpmc_usc,1401,0,0,6,0,0,0,University of Southern California
8,uamc_banner,2099,22,3672,185,220,11032,69,Banner Health
9,saou_uab,2465,4,107523,0,1,0,0,University of Alabama at Birmingham


In [37]:
sites_success.to_csv("data\\duplicates.csv")

# 20.Dataframe (row for each hpo_id) Condition_occurrence table, condition_source_concept_id field

In [38]:
 condition_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_total_row
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_well_defined_row
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.condition_concept_id
            WHERE 
                 t3.domain_id="Condition" and t3.standard_concept="S"
            GROUP BY
                1
        ),
        
        
        data3 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_total_zero
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.condition_concept_id
            WHERE 
                 (t3.concept_id=0 or t3.concept_id is null)
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        condition_well_defined_row,
        condition_total_row,
        round(100*(condition_well_defined_row/condition_total_row),1) as condition_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    LEFT OUTER JOIN
        data3
    ON
        data1.src_hpo_id=data3.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
condition_concept_df.shape

(32, 4)

In [39]:
condition_concept_df=condition_concept_df.fillna(0)
condition_concept_df

,src_hpo_id,condition_well_defined_row,condition_total_row,condition_succes_rate
0,uamc_banner,270.0,151401,0.2
1,trans_am_spectrum,263956.0,263956,100.0
2,tach_hfhs,1091225.0,1091225,100.0
3,syhc,162158.0,162158,100.0
4,seec_ufl,277332.0,277596,99.9
5,seec_morehouse,21752.0,21759,100.0
6,seec_miami,356721.0,356736,100.0
7,seec_emory,122724.0,122724,100.0
8,saou_uab,3411.0,577283,0.6
9,pitt,2275187.0,2275187,100.0


# 21.Dataframe (row for each hpo_id) Procedure_occurrence table, procedure_source_concept_id field

In [40]:
procedure_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_total_row
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_well_defined_row
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.procedure_source_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Procedure"
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        procedure_well_defined_row,
        procedure_total_row,
        round(100*(procedure_well_defined_row/procedure_total_row),1) as procedure_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
procedure_concept_df.shape

(32, 4)

In [41]:
procedure_concept_df=procedure_concept_df.fillna(0)
procedure_concept_df

,src_hpo_id,procedure_well_defined_row,procedure_total_row,procedure_succes_rate
0,uamc_banner,218217.0,675798,32.3
1,trans_am_spectrum,84271.0,84271,100.0
2,syhc,91888.0,92105,99.8
3,seec_ufl,0.0,73604,0.0
4,seec_morehouse,124946.0,246882,50.6
5,seec_miami,336280.0,440386,76.4
6,seec_emory,1099.0,10383,10.6
7,saou_uab,116857.0,151228,77.3
8,pitt,598269.0,730117,81.9
9,nyc_hh,11670.0,11670,100.0


# 22.Dataframe (row for each hpo_id) Drug_exposures table, drug_source_concept_id  field

In [42]:
drug_exp_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.drug_source_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Drug"
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        drug_exp_well_defined_row,
        drug_exp_total_row,
        round(100*(drug_exp_well_defined_row/drug_exp_total_row),1) as drug_exp_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
drug_exp_concept_df.shape

(33, 4)

In [43]:
drug_exp_concept_df=drug_exp_concept_df.fillna(0)
drug_exp_concept_df

,src_hpo_id,drug_exp_well_defined_row,drug_exp_total_row,drug_exp_succes_rate
0,uamc_banner,94782.0,405745,23.4
1,trans_am_spectrum,81963.0,82083,99.9
2,tach_hfhs,459345.0,812580,56.5
3,syhc,0.0,66351,0.0
4,seec_ufl,831.0,74689,1.1
5,seec_morehouse,3351.0,90636,3.7
6,seec_miami,62511.0,86662,72.1
7,seec_emory,54559.0,54642,99.8
8,saou_uab,590298.0,716779,82.4
9,pitt,727620.0,2938146,24.8


# 23.Dataframe (row for each hpo_id) Observation table, Observation_source_concept_id field



In [44]:
observation_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_total_row
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_well_defined_row
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.observation_source_concept_id 
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Observation"
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        observation_well_defined_row,
        observation_total_row,
        round(100*(observation_well_defined_row/observation_total_row),1) as observation_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
observation_concept_df.shape

(34, 4)

In [45]:
observation_concept_df=observation_concept_df.fillna(0)
observation_concept_df

,src_hpo_id,observation_well_defined_row,observation_total_row,observation_succes_rate
0,uamc_banner,0.0,154695,0.0
1,trans_am_spectrum,0.0,43652,0.0
2,tach_hfhs,64411.0,189033,34.1
3,syhc,0.0,37087,0.0
4,seec_ufl,0.0,35847,0.0
5,seec_morehouse,34084.0,39888,85.4
6,seec_miami,2204927.0,2461772,89.6
7,seec_emory,0.0,22283,0.0
8,saou_uab,0.0,601863,0.0
9,pitt,14162.0,448515,3.2


In [46]:
datas=[
       procedure_concept_df,drug_exp_concept_df,observation_concept_df
]

In [47]:
master_df=condition_concept_df

In [48]:
for filename in datas:
    master_df = pd.merge(master_df,filename,on='src_hpo_id',how='outer')

In [49]:
master_df

,src_hpo_id,condition_well_defined_row,condition_total_row,condition_succes_rate,procedure_well_defined_row,procedure_total_row,procedure_succes_rate,drug_exp_well_defined_row,drug_exp_total_row,drug_exp_succes_rate,observation_well_defined_row,observation_total_row,observation_succes_rate
0,uamc_banner,270.0,151401.0,0.2,218217.0,675798.0,32.3,94782.0,405745.0,23.4,0.0,154695,0.0
1,trans_am_spectrum,263956.0,263956.0,100.0,84271.0,84271.0,100.0,81963.0,82083.0,99.9,0.0,43652,0.0
2,tach_hfhs,1091225.0,1091225.0,100.0,NaN,NaN,NaN,459345.0,812580.0,56.5,64411.0,189033,34.1
3,syhc,162158.0,162158.0,100.0,91888.0,92105.0,99.8,0.0,66351.0,0.0,0.0,37087,0.0
4,seec_ufl,277332.0,277596.0,99.9,0.0,73604.0,0.0,831.0,74689.0,1.1,0.0,35847,0.0
5,seec_morehouse,21752.0,21759.0,100.0,124946.0,246882.0,50.6,3351.0,90636.0,3.7,34084.0,39888,85.4
6,seec_miami,356721.0,356736.0,100.0,336280.0,440386.0,76.4,62511.0,86662.0,72.1,2204927.0,2461772,89.6
7,seec_emory,122724.0,122724.0,100.0,1099.0,10383.0,10.6,54559.0,54642.0,99.8,0.0,22283,0.0
8,saou_uab,3411.0,577283.0,0.6,116857.0,151228.0,77.3,590298.0,716779.0,82.4,0.0,601863,0.0
9,pitt,2275187.0,2275187.0,100.0,598269.0,730117.0,81.9,727620.0,2938146.0,24.8,14162.0,448515,3.2


In [50]:
source=pd.merge(master_df,site_df,how='outer',on='src_hpo_id')
source=source.fillna("No Data")
source.to_csv("data\\source.csv")

# 16.Dataframe (row for each hpo_id) Condition_occurrence table, condition_concept_id field

In [51]:
condition_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_total_row
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            GROUP BY
                1
        ),


        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_well_defined_row
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.condition_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Condition"
            GROUP BY
                1
        ),
        
        data3 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_total_zeros_or_null
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            WHERE 
                (t1.condition_concept_id=0 or t1.condition_concept_id IS NULL)
            GROUP BY
                1
        ),
        
                data4 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS condition_total_null
            FROM
               `{}.unioned_ehr_condition_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_condition_occurrence`)  AS t2
            ON
                t1.condition_occurrence_id=t2.condition_occurrence_id
            WHERE 
                t1.condition_concept_id IS NULL
            GROUP BY
                1
        )


    SELECT
        data1.src_hpo_id,
        condition_well_defined_row,
        condition_total_row,
        condition_total_zeros_or_null,
        condition_total_null,
        round(100*(condition_well_defined_row/condition_total_row),1) as condition_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    LEFT OUTER JOIN
        data3
    ON
        data1.src_hpo_id=data3.src_hpo_id
    LEFT OUTER JOIN
        data4
    ON
        data1.src_hpo_id=data4.src_hpo_id
    ORDER BY
        4 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
condition_concept_df.shape

(32, 6)

In [52]:
condition_concept_df=condition_concept_df.fillna(0)
condition_concept_df

,src_hpo_id,condition_well_defined_row,condition_total_row,condition_total_zeros_or_null,condition_total_null,condition_succes_rate
0,aouw_uwh,8881242.0,9468137,586895.0,0,93.8
1,nyc_cu,1720479.0,2086163,365684.0,0,82.5
2,uamc_banner,270.0,151401,151120.0,0,0.2
3,cpmc_ucd,641543.0,771216,129673.0,0,83.2
4,nec_phs,3402374.0,3500964,98590.0,0,97.2
5,ipmc_nu,773298.0,775980,2682.0,0,99.7
6,cpmc_usc,299414.0,303487,2354.0,0,98.7
7,aouw_mcri,1875243.0,1876917,1674.0,0,99.9
8,cpmc_ucsf,549.0,548575,366.0,0,0.1
9,ipmc_northshore,118181.0,118395,214.0,0,99.8


# 17.Dataframe (row for each hpo_id) Procedure_occurrence table, procedure_concept_id field

In [53]:
procedure_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_total_row
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_well_defined_row
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.procedure_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Procedure"
            GROUP BY
                1
        ),
        
        data3 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_total_zero_null
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE 
                 (t1.procedure_concept_id=0 or t1.procedure_concept_id IS NULL)
            GROUP BY
                1
        ),
        
                data4 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS procedure_total_null
            FROM
               `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
            ON
                t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE 
                 t1.procedure_concept_id IS NULL
            GROUP BY
                1
        )
        

    SELECT
        data1.src_hpo_id,
        procedure_well_defined_row,
        procedure_total_zero_null,
        procedure_total_null,
        procedure_total_row,
        round(100*(procedure_well_defined_row/procedure_total_row),1) as procedure_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    LEFT OUTER JOIN
        data3
    ON
        data1.src_hpo_id=data3.src_hpo_id
    LEFT OUTER JOIN
        data4
    ON
        data1.src_hpo_id=data4.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
procedure_concept_df.shape

(32, 6)

In [54]:
procedure_concept_df=procedure_concept_df.fillna(0)
procedure_concept_df

,src_hpo_id,procedure_well_defined_row,procedure_total_zero_null,procedure_total_null,procedure_total_row,procedure_succes_rate
0,uamc_banner,289167,385319.0,0,675798,42.8
1,trans_am_spectrum,84271,0.0,0,84271,100.0
2,syhc,92105,0.0,0,92105,100.0
3,seec_ufl,73604,0.0,0,73604,100.0
4,seec_morehouse,129837,116937.0,0,246882,52.6
5,seec_miami,367374,1153.0,0,440386,83.4
6,seec_emory,10379,2.0,0,10383,100.0
7,saou_uab,116857,0.0,0,151228,77.3
8,pitt,728835,0.0,0,730117,99.8
9,nyc_hh,11670,0.0,0,11670,100.0


# 18.Dataframe (row for each hpo_id) Drug_exposures table, drug_concept_id field

In [55]:
drug_exp_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.drug_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Drug"
            GROUP BY
                1
        ),

        data3 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_total_zero_null
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.drug_concept_id
            WHERE 
                 (t1.drug_concept_id=0 OR t1.drug_concept_id IS NULL)
            GROUP BY
                1
        ),
        
                data4 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS drug_exp_total_null
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE 
                 t1.drug_concept_id IS NULL
            GROUP BY
                1
        )
        
    SELECT
        data1.src_hpo_id,
        drug_exp_well_defined_row,
        drug_exp_total_zero_null,
        drug_exp_total_null,
        drug_exp_total_row,
        round(100*(drug_exp_well_defined_row/drug_exp_total_row),1) as drug_exp_succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    LEFT OUTER JOIN
        data3
    ON
        data1.src_hpo_id=data3.src_hpo_id
    LEFT OUTER JOIN
        data4
    ON
        data1.src_hpo_id=data4.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
drug_exp_concept_df.shape

(33, 6)

In [56]:

drug_exp_concept_df=drug_exp_concept_df.fillna(0)
drug_exp_concept_df

,src_hpo_id,drug_exp_well_defined_row,drug_exp_total_zero_null,drug_exp_total_null,drug_exp_total_row,drug_exp_succes_rate
0,uamc_banner,342885,57958.0,0,405745,84.5
1,trans_am_spectrum,81963,0.0,0,82083,99.9
2,tach_hfhs,726919,79370.0,0,812580,89.5
3,syhc,66351,0.0,0,66351,100.0
4,seec_ufl,72960,284.0,0,74689,97.7
5,seec_morehouse,3362,87271.0,0,90636,3.7
6,seec_miami,73704,12738.0,0,86662,85.0
7,seec_emory,54626,5.0,0,54642,100.0
8,saou_uab,590298,0.0,0,716779,82.4
9,pitt,2898133,0.0,0,2938146,98.6


# 19.Dataframe (row for each hpo_id) Observation table, Observation_concept_id field


In [57]:
observation_concept_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_total_row
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_well_defined_row
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.observation_concept_id
            WHERE 
                 t3.standard_concept="S" and t3.domain_id="Observation"
            GROUP BY
                1
        ),

        data3 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_total_zero_missing
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.observation_concept_id
            WHERE 
                 (t1.observation_concept_id=0 OR t1.observation_concept_id IS NULL)
            GROUP BY
                1
        ),
        
            data4 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS observation_total_missing
            FROM
               `{}.unioned_ehr_observation` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_observation`)  AS t2
            ON
                t1.observation_id=t2.observation_id
            WHERE 
              t1.observation_concept_id IS NULL
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        observation_total_zero_missing,
        observation_total_missing,
        observation_well_defined_row,
        observation_total_row,
        round(100*(observation_well_defined_row/observation_total_row),1) as succes_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    LEFT OUTER JOIN
        data3
    ON
        data1.src_hpo_id=data3.src_hpo_id
    LEFT OUTER JOIN
        data4
    ON
        data1.src_hpo_id=data4.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
observation_concept_df.shape

(34, 6)

In [58]:
observation_concept_df=observation_concept_df.fillna(0)
observation_concept_df

,src_hpo_id,observation_total_zero_missing,observation_total_missing,observation_well_defined_row,observation_total_row,succes_rate
0,uamc_banner,0.0,0,154695,154695,100.0
1,trans_am_spectrum,0.0,0,43652,43652,100.0
2,tach_hfhs,0.0,0,189033,189033,100.0
3,syhc,0.0,0,5792,37087,15.6
4,seec_ufl,0.0,0,35847,35847,100.0
5,seec_morehouse,0.0,0,39888,39888,100.0
6,seec_miami,0.0,0,2461772,2461772,100.0
7,seec_emory,0.0,0,22283,22283,100.0
8,saou_uab,0.0,0,509552,601863,84.7
9,pitt,0.0,0,448479,448515,100.0


## Sites combined

In [59]:
datas=[
drug_exp_concept_df,
procedure_concept_df,
condition_concept_df
]

master_df=observation_concept_df

for filename in datas:
    master_df = pd.merge(master_df,filename,on='src_hpo_id',how='outer')

master_df

succes_rate=pd.merge(master_df,site_df,how='outer',on='src_hpo_id')
succes_rate

,src_hpo_id,observation_total_zero_missing,observation_total_missing,observation_well_defined_row,observation_total_row,succes_rate,drug_exp_well_defined_row,drug_exp_total_zero_null,drug_exp_total_null,drug_exp_total_row,drug_exp_succes_rate,procedure_well_defined_row,procedure_total_zero_null,procedure_total_null,procedure_total_row,procedure_succes_rate,condition_well_defined_row,condition_total_row,condition_total_zeros_or_null,condition_total_null,condition_succes_rate,HPO
0,uamc_banner,0.0,0.0,154695.0,154695.0,100.0,342885.0,57958.0,0.0,405745.0,84.5,289167.0,385319.0,0.0,675798.0,42.8,270.0,151401.0,151120.0,0.0,0.2,Banner Health
1,trans_am_spectrum,0.0,0.0,43652.0,43652.0,100.0,81963.0,0.0,0.0,82083.0,99.9,84271.0,0.0,0.0,84271.0,100.0,263956.0,263956.0,0.0,0.0,100.0,Spectrum Health
2,tach_hfhs,0.0,0.0,189033.0,189033.0,100.0,726919.0,79370.0,0.0,812580.0,89.5,NaN,NaN,NaN,NaN,NaN,1091225.0,1091225.0,0.0,0.0,100.0,Henry Ford Health System
3,syhc,0.0,0.0,5792.0,37087.0,15.6,66351.0,0.0,0.0,66351.0,100.0,92105.0,0.0,0.0,92105.0,100.0,162158.0,162158.0,0.0,0.0,100.0,San Ysidro Health Center
4,seec_ufl,0.0,0.0,35847.0,35847.0,100.0,72960.0,284.0,0.0,74689.0,97.7,73604.0,0.0,0.0,73604.0,100.0,277332.0,277596.0,0.0,0.0,99.9,University of Florida
5,seec_morehouse,0.0,0.0,39888.0,39888.0,100.0,3362.0,87271.0,0.0,90636.0,3.7,129837.0,116937.0,0.0,246882.0,52.6,21752.0,21759.0,0.0,0.0,100.0,SouthEast Enrollment Center Morehouse
6,seec_miami,0.0,0.0,2461772.0,2461772.0,100.0,73704.0,12738.0,0.0,86662.0,85.0,367374.0,1153.0,0.0,440386.0,83.4,356721.0,356736.0,3.0,0.0,100.0,SouthEast Enrollment Center Miami
7,seec_emory,0.0,0.0,22283.0,22283.0,100.0,54626.0,5.0,0.0,54642.0,100.0,10379.0,2.0,0.0,10383.0,100.0,122724.0,122724.0,0.0,0.0,100.0,SouthEast Enrollment Center Emory
8,saou_uab,0.0,0.0,509552.0,601863.0,84.7,590298.0,0.0,0.0,716779.0,82.4,116857.0,0.0,0.0,151228.0,77.3,3411.0,577283.0,0.0,0.0,0.6,University of Alabama at Birmingham
9,pitt,0.0,0.0,448479.0,448515.0,100.0,2898133.0,0.0,0.0,2938146.0,98.6,728835.0,0.0,0.0,730117.0,99.8,2275187.0,2275187.0,0.0,0.0,100.0,University of Pittsburgh


In [60]:
succes_rate=succes_rate.fillna("No Data")
succes_rate

succes_rate.to_csv("data\\concept.csv")